In [1]:
import pandas as pd
import numpy as np 
import scipy.stats as stats
import matplotlib.pyplot as plt


student_df = pd.read_csv('/content/drive/MyDrive/DataMining/Datasets/submission_1.csv')
col = student_df.columns
column_name = col.to_list()

column_name.remove('class')
print(column_name)

['mfcc_m1', 'mfcc_m2', 'mfcc_m3', 'mfcc_m4', 'mfcc_m5', 'mfcc_m6', 'harmonics_m1', 'harmonics_m2', 'harmonics_m3', 'harmonics_m4', 'harmonics_m5', 'harmonics_m6', 'perceptrual_m1', 'perceptrual_m2', 'perceptrual_m3', 'perceptrual_m4', 'perceptrual_m5', 'perceptrual_m6', 'zero_crossing_rate_m1', 'zero_crossing_rate_m2', 'zero_crossing_rate_m3', 'zero_crossing_rate_m4', 'zero_crossing_rate_m5', 'zero_crossing_rate_m6', 'spectral_rollof_m1', 'spectral_rollof_m2', 'spectral_rollof_m3', 'spectral_rollof_m4', 'spectral_rollof_m5', 'spectral_rollof_m6', 'spectral_flux_m1', 'spectral_flux_m2', 'spectral_flux_m3', 'spectral_flux_m4', 'spectral_flux_m5', 'spectral_flux_m6']


In [2]:
student_df.head()

,mfcc_m1,mfcc_m2,mfcc_m3,mfcc_m4,mfcc_m5,mfcc_m6,harmonics_m1,harmonics_m2,harmonics_m3,harmonics_m4,...,spectral_rollof_m4,spectral_rollof_m5,spectral_rollof_m6,spectral_flux_m1,spectral_flux_m2,spectral_flux_m3,spectral_flux_m4,spectral_flux_m5,spectral_flux_m6,class
0,-267.24225,137.70125,-11.551569,-6.795150,66.951870,4482.5530,-0.112410,0.107032,-0.000010,-0.001025,...,2691.650391,589.766689,3.478247e+05,0.0,9.985063,1.743083,1.080145,2.030075,4.121204,Mild
1,-235.51150,95.75791,-14.844896,-7.628602,57.059990,3255.8425,-0.367907,0.393908,-0.000004,-0.000686,...,3736.010742,2310.012162,5.336156e+06,0.0,13.284759,2.316962,1.491382,2.516871,6.334639,Moderate
2,-213.29170,110.82733,-12.964460,-8.223975,54.339092,2952.7370,-0.554416,0.582450,-0.000005,0.000000,...,3262.280273,1521.084813,2.313699e+06,0.0,8.369490,1.889303,1.481027,1.721631,2.964013,Moderate
3,-258.55457,121.32260,-15.070149,-8.624528,63.593155,4044.0890,-0.327833,0.305298,0.000005,0.000000,...,3267.663574,1427.159000,2.036783e+06,0.0,9.896101,1.719413,0.946093,1.900773,3.612939,Extreme
4,-204.03584,112.08085,-13.763713,-10.227917,52.844110,2792.4998,-0.501883,0.556520,-0.000022,0.000000,...,3391.479492,752.747604,5.666290e+05,0.0,8.118092,1.823508,1.260924,1.791249,3.208574,Moderate


**Open**

In [3]:
import pandas as pd
import random


for x in column_name:
  # two variables of interest
  sample_df = student_df[[x, 'class']]
  groups = sample_df.groupby('class').count().reset_index()
  # groups.plot(kind='bar',x='major',y='salary')
  print(groups)
  data = [['Between Groups', '', '', '', '', '', ''], ['Within Groups', '', '', '', '', '', ''], ['Total', '', '', '', '', '', '']] 
  anova_table = pd.DataFrame(data, columns = ['Source of Variation', 'SS', 'df', 'MS', 'F', 'P-value', 'F crit']) 
  anova_table.set_index('Source of Variation', inplace = True)
  # calculate SSTR and update anova table
  x_bar = sample_df[x].mean()
  SSTR = sample_df.groupby('class').count() * (sample_df.groupby('class').mean() - x_bar)**2
  anova_table['SS']['Between Groups'] = SSTR[x].sum()
  # calculate SSE and update anova table
  SSE = (sample_df.groupby('class').count() - 1) * sample_df.groupby('class').std()**2
  anova_table['SS']['Within Groups'] = SSE[x].sum()
  # calculate SSTR and update anova table
  SSTR = SSTR[x].sum() + SSE[x].sum()
  anova_table['SS']['Total'] = SSTR

  # update degree of freedom
  anova_table['df']['Between Groups'] = sample_df['class'].nunique() - 1
  anova_table['df']['Within Groups'] = sample_df.shape[0] - sample_df['class'].nunique()
  anova_table['df']['Total'] = sample_df.shape[0] - 1

  # calculate MS
  anova_table['MS'] = anova_table['SS'] / anova_table['df']

  # calculate F 
  F = anova_table['MS']['Between Groups'] / anova_table['MS']['Within Groups']
  anova_table['F']['Between Groups'] = F

  # p-value
  anova_table['P-value']['Between Groups'] = 1 - stats.f.cdf(F, anova_table['df']['Between Groups'], anova_table['df']['Within Groups'])

  # F critical 
  alpha = 0.05
  # possible types "right-tailed, left-tailed, two-tailed"
  tail_hypothesis_type = "two-tailed"
  if tail_hypothesis_type == "two-tailed":
      alpha /= 2
  anova_table['F crit']['Between Groups'] = stats.f.ppf(1-alpha, anova_table['df']['Between Groups'], anova_table['df']['Within Groups'])

  # Final ANOVA Table
  print("For the variable {} :".format(x))
  print('')
  anova_table

  # The p-value approach
  print("Approach 1: The p-value approach to hypothesis testing in the decision rule")
  conclusion = "Failed to reject the null hypothesis."
  if anova_table['P-value']['Between Groups'] <= alpha:
      conclusion = "Null Hypothesis is rejected."
  print("F-score is:", anova_table['F']['Between Groups'], " and p value is:", anova_table['P-value']['Between Groups'])    
  print(conclusion)
      
  # The critical value approach
  print('\n')
  print("Approach 2: The critical value approach to hypothesis testing in the decision rule")
  conclusion = "Failed to reject the null hypothesis."
  if anova_table['F']['Between Groups'] > anova_table['F crit']['Between Groups']:
      conclusion = "Null Hypothesis is rejected."
  print("F-score is:", anova_table['F']['Between Groups'], " and critical value is:", anova_table['F crit']['Between Groups'])
  print(conclusion)
  print('\n----------------------------------------------------------------------------------------------------------------')




      class  mfcc_m1
0   Extreme      203
1      Mild      212
2   Minimal      112
3  Moderate      144
For the variable mfcc_m1 :

Approach 1: The p-value approach to hypothesis testing in the decision rule
F-score is: 31.304841651552653  and p value is: 1.1102230246251565e-16
Null Hypothesis is rejected.


Approach 2: The critical value approach to hypothesis testing in the decision rule
F-score is: 31.304841651552653  and critical value is: 3.1357112788353287
Null Hypothesis is rejected.

----------------------------------------------------------------------------------------------------------------
      class  mfcc_m2
0   Extreme      203
1      Mild      212
2   Minimal      112
3  Moderate      144
For the variable mfcc_m2 :

Approach 1: The p-value approach to hypothesis testing in the decision rule
F-score is: 75.94245009920502  and p value is: 1.1102230246251565e-16
Null Hypothesis is rejected.


Approach 2: The critical value approach to hypothesis testing in the decision r

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:34: RuntimeWarning: invalid value encountered in double_scalars
